In [64]:
import pandas as pd
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from sklearn import metrics
from sklearn import ensemble
from statsmodels.regression import linear_model
from sklearn.preprocessing import PolynomialFeatures
from yellowbrick.regressor import ResidualsPlot
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy.stats.mstats import winsorize
import warnings
warnings.filterwarnings('ignore')

In [53]:
data = pd.read_csv('Train.csv')

In [54]:
#function for dropping columns
def drop_col(df,col_list):
    for col in col_list:
        if col in df.columns:
            df.drop(col, axis = 1, inplace=True)
    return df

In [55]:
not_req = ['ID','region_code','personal_loan_active','vehicle_loan_active','personal_loan_closed','vehicle_loan_closed','loan_enq']

numerical_cols = ['age', 'cc_cons_apr', 'dc_cons_apr', 'cc_cons_may', 'dc_cons_may',
       'cc_cons_jun', 'dc_cons_jun', 'cc_count_apr', 'cc_count_may',
       'cc_count_jun', 'dc_count_apr', 'dc_count_may', 'dc_count_jun',
       'card_lim', 'investment_1', 'investment_2', 'investment_3',
       'investment_4', 'debit_amount_apr', 'credit_amount_apr',
       'debit_count_apr', 'credit_count_apr', 'max_credit_amount_apr',
       'debit_amount_may', 'credit_amount_may', 'credit_count_may',
       'debit_count_may', 'max_credit_amount_may', 'debit_amount_jun',
       'credit_amount_jun', 'credit_count_jun', 'debit_count_jun',
       'max_credit_amount_jun', 'emi_active']

In [56]:
#removing unwanted columns 
data = drop_col(data, not_req)

In [57]:
#encoding categorical data using one hot encoding
data = pd.get_dummies(data=data, columns=['account_type', 'gender'])
data.head(5)

,age,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,...,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,emi_active,cc_cons,account_type_current,account_type_saving,gender_F,gender_M
0,35,24893.00,378.00,10288.00,29664.00,16291.40,11432.00,2.0,26.0,10.0,...,12761.0,2.0,65.0,50836.0,1674.09,20014.0,1,0,0,1
1,35,18941.62,966.00,20672.00,287.00,4217.00,885.00,1.0,7.0,13.0,...,76206.0,21.0,63.0,23226.0,13043.34,10173.0,1,0,0,1
2,55,5678.87,2724.00,1964.50,3933.11,23956.25,5168.00,43.0,32.0,102.0,...,75283.0,7.0,1.0,27036.0,25375.27,16095.0,1,0,0,1
3,29,30489.50,1236.00,12609.88,9138.14,17521.00,13650.30,53.0,1.0,50.0,...,68708.0,21.0,83.0,43037.0,3544.33,7707.0,1,0,1,0
4,34,7441.40,6906.04,4364.00,1939.00,2121.00,6829.18,67.0,47.0,82.0,...,80140.0,8.0,32.0,32044.0,12780.44,96408.0,1,0,0,1


In [58]:
cat_cols = ['account_type_current','account_type_saving','gender_F','gender_M']

In [59]:
X = data.drop(['cc_cons'], axis=1) 
y = data['cc_cons']

In [60]:
model = linear_model.OLS(y, X).fit()

In [61]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                cc_cons   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.8009
Date:                Sat, 29 Feb 2020   Prob (F-statistic):              0.796
Time:                        12:14:41   Log-Likelihood:            -1.9361e+05
No. Observations:               15000   AIC:                         3.873e+05
Df Residuals:                   14963   BIC:                         3.876e+05
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
age                     -21.9842     26.079     -0.843      0.399     -73.103      29.135
cc_cons_apr               0.0234      0.032      0.737      0.461      -0.039       0.085
dc_cons_apr              -0.0014      0.054     -0.026      0.979      -0.107       0.105
cc_cons_may               0.0788      0.030      2.671      0.008       0.021       0.137
dc_cons_may              -0.0055      0.042     -0.130      0.897      -0.088       0.077
cc_cons_jun              -0.0024      0.043     -0.054      0.957      -0.087       0.082
dc_cons_jun               0.0202      0.060      0.336      0.737      -0.098       0.138
cc_count_apr             -9.5503      8.194     -1.166      0.244     -25.611       6.510
cc_count_may             31.0012     16.130      1.922      0.055      -0.616      62.618
cc_count_jun             -2.4185     16.867     -0.143      0.886     -35.481      30.643
dc_count_apr             -4.3583     41.248     -0.106      0.916     -85.209      76.492
dc_count_may             44.3766     46.763      0.949      0.343     -47.286     136.039
dc_count_jun            -20.6355     27.301     -0.756      0.450     -74.148      32.877
card_lim                 -0.0018      0.004     -0.407      0.684      -0.011       0.007
investment_1              0.0001      0.000      0.545      0.586      -0.000       0.001
investment_2              0.0020      0.002      1.033      0.302      -0.002       0.006
investment_3             -0.0128      0.009     -1.410      0.158      -0.031       0.005
investment_4              0.0238      0.108      0.219      0.826      -0.189       0.236
debit_amount_apr          0.0036      0.007      0.506      0.613      -0.010       0.017
credit_amount_apr         0.0029      0.004      0.716      0.474      -0.005       0.011
debit_count_apr          -5.2225     26.016     -0.201      0.841     -56.218      45.773
credit_count_apr        -10.5259     63.033     -0.167      0.867    -134.079     113.027
max_credit_amount_apr    -0.0017      0.004     -0.415      0.678      -0.010       0.006
debit_amount_may         -0.0010      0.008     -0.124      0.901      -0.016       0.014
credit_amount_may        -0.0016      0.004     -0.354      0.723      -0.010       0.007
credit_count_may         79.5560     47.427      1.677      0.093     -13.407     172.519
debit_count_may          -0.4672     19.711     -0.024      0.981     -39.102      38.168
max_credit_amount_may     0.0036      0.004      0.913      0.361      -0.004       0.011
debit_amount_jun         -0.0035      0.005     -0.678      0.498      -0.014       0.007
credit_amount_jun        -0.0085      0.005     -1.586      0.113      -0.019       0.002
credit_count_jun         18.3717     29.360      0.626      0.531     -39.177      75.920
debit_count_jun        

### Most of the variables have a high p-value. They stand-alone do not contribute anything to the predictions.

In [65]:
interaction = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)

In [66]:
X_inter = interaction.fit_transform(X)

In [67]:
X_inter

array([[3.500000e+01, 2.489300e+04, 3.780000e+02, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [3.500000e+01, 1.894162e+04, 9.660000e+02, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [5.500000e+01, 5.678870e+03, 2.724000e+03, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       ...,
       [3.700000e+01, 4.148980e+03, 9.487400e+02, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [2.200000e+02, 1.155740e+05, 7.191050e+03, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [6.600000e+01, 7.375670e+03, 3.068000e+02, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00]])

In [68]:
X.columns

Index(['age', 'cc_cons_apr', 'dc_cons_apr', 'cc_cons_may', 'dc_cons_may',
       'cc_cons_jun', 'dc_cons_jun', 'cc_count_apr', 'cc_count_may',
       'cc_count_jun', 'dc_count_apr', 'dc_count_may', 'dc_count_jun',
       'card_lim', 'investment_1', 'investment_2', 'investment_3',
       'investment_4', 'debit_amount_apr', 'credit_amount_apr',
       'debit_count_apr', 'credit_count_apr', 'max_credit_amount_apr',
       'debit_amount_may', 'credit_amount_may', 'credit_count_may',
       'debit_count_may', 'max_credit_amount_may', 'debit_amount_jun',
       'credit_amount_jun', 'credit_count_jun', 'debit_count_jun',
       'max_credit_amount_jun', 'emi_active', 'account_type_current',
       'account_type_saving', 'gender_F', 'gender_M'],
      dtype='object')

In [69]:
x_interaction = pd.DataFrame(X_inter, columns = ['age','cc_cons_apr','dc_cons_apr','cc_cons_may','dc_cons_may','cc_cons_jun',
        'dc_cons_jun','cc_count_apr','cc_count_may',
       'cc_count_jun', 'dc_count_apr', 'dc_count_may', 'dc_count_jun',
       'card_lim', 'investment_1', 'investment_2', 'investment_3',
       'investment_4', 'debit_amount_apr', 'credit_amount_apr',
       'debit_count_apr', 'credit_count_apr', 'max_credit_amount_apr',
       'debit_amount_may', 'credit_amount_may', 'credit_count_may',
       'debit_count_may', 'max_credit_amount_may', 'debit_amount_jun',
       'credit_amount_jun', 'credit_count_jun', 'debit_count_jun',
       'max_credit_amount_jun', 'emi_active', 'account_type_current',
       'account_type_saving', 'gender_F', 'gender_M',
        'age:cc_cons_apr','age:dc_cons_apr','age:cc_cons_may','age:dc_cons_may','age:cc_cons_jun',
        'age:dc_cons_jun','age:cc_count_apr','age:cc_count_may',
       'age:cc_count_jun', 'age:dc_count_apr', 'age:dc_count_may', 'age:dc_count_jun',
       'age:card_lim', 'age:investment_1', 'age:investment_2', 'age:investment_3',
       'age:investment_4', 'age:debit_amount_apr', 'age:credit_amount_apr',
       'age:debit_count_apr', 'age:credit_count_apr', 'age:max_credit_amount_apr',
       'age:debit_amount_may', 'age:credit_amount_may', 'age:credit_count_may',
       'age:debit_count_may', 'age:max_credit_amount_may', 'age:debit_amount_jun',
       'age:credit_amount_jun', 'age:credit_count_jun', 'age:debit_count_jun',
       'age:max_credit_amount_jun', 'age:emi_active', 'age:account_type_current',
       'age:account_type_saving', 'age:gender_F', 'age:gender_M',
        'cc_cons_apr:dc_cons_apr','cc_cons_apr:cc_cons_may','cc_cons_apr:dc_cons_may','cc_cons_apr:cc_cons_jun',
        'cc_cons_apr:dc_cons_jun','cc_cons_apr:cc_count_apr','cc_cons_apr:cc_count_may',
       'cc_cons_apr:cc_count_jun', 'cc_cons_apr:dc_count_apr', 'cc_cons_apr:dc_count_may', 'cc_cons_apr:dc_count_jun',
       'cc_cons_apr:card_lim', 'cc_cons_apr:investment_1', 'cc_cons_apr:investment_2', 'cc_cons_apr:investment_3',
       'cc_cons_apr:investment_4', 'cc_cons_apr:debit_amount_apr', 'cc_cons_apr:credit_amount_apr',
       'cc_cons_apr:debit_count_apr', 'cc_cons_apr:credit_count_apr', 'cc_cons_apr:max_credit_amount_apr',
       'cc_cons_apr:debit_amount_may', 'cc_cons_apr:credit_amount_may', 'cc_cons_apr:credit_count_may',
       'cc_cons_apr:debit_count_may', 'cc_cons_apr:max_credit_amount_may', 'cc_cons_apr:debit_amount_jun',
       'cc_cons_apr:credit_amount_jun', 'cc_cons_apr:credit_count_jun', 'cc_cons_apr:debit_count_jun',
       'cc_cons_apr:max_credit_amount_jun', 'cc_cons_apr:emi_active', 'cc_cons_apr:account_type_current',
       'cc_cons_apr:account_type_saving', 'cc_cons_apr:gender_F', 'cc_cons_apr:gender_M',
        'dc_cons_apr:cc_cons_may','dc_cons_apr:dc_cons_may','dc_cons_apr:cc_cons_jun',
        'dc_cons_apr:dc_cons_jun','dc_cons_apr:cc_count_apr','dc_cons_apr:cc_count_may',
       'dc_cons_apr:cc_count_jun', 'dc_cons_apr:dc_count_apr', 'dc_cons_apr:dc_count_may', 'dc_cons_apr:dc_count_jun',
       'dc_cons_apr:card_lim', 'dc_cons_apr:investment_1', 'dc_cons_apr:investment_2', 'dc_cons_apr:investment_3',
       'dc_cons_apr:investment_4', 'dc_cons_apr:debit_amount_apr', 'dc_cons_apr:credit_amount_apr',
       'dc_cons_apr:debit_count_apr', 'dc_cons_apr:credit_count_apr', 'dc_cons_apr:max_credit_amount_apr',
       'dc_cons_apr:debit_amount_may', 'dc_cons_apr:credit_amount_may', 'dc_cons_apr:credit_count_may',
       'dc_cons_apr:debit_count_may', 'dc_cons_apr:max_credit_amount_may', 'dc_cons_apr:debit_amount_jun',
       'dc_cons_apr:credit_amount_jun', 'dc_cons_apr:credit_count_jun', 'dc_cons_apr:debit_count_jun',
       'dc_cons_apr:max_credit_amount_jun', 'dc_cons_apr:emi_active', 'dc_cons_apr:account_type_current',
       'dc_cons_apr:account_type_saving', 'dc_cons_apr:gender_F', 'dc_cons_apr:gender_M',
        'cc_cons_may:dc_cons_may','cc_cons_may:cc_cons_jun',
        'cc_cons_may:dc_cons_jun','cc_cons_may:cc_count_apr','cc_cons_may:cc_count_may',
       'cc_cons_may:cc_count_jun', 'cc_cons_may:dc_count_apr', 'cc_cons_may:dc_count_may', 'cc_cons_may:dc_count_jun',
       'cc_cons_may:card_lim', 'cc_cons_may:investment_1', 'cc_cons_may:investment_2', 'cc_cons_may:investment_3',
       'cc_cons_may:investment_4', 'cc_cons_may:debit_amount_apr', 'cc_cons_may:credit_amount_apr',
       'cc_cons_may:debit_count_apr', 'cc_cons_may:credit_count_apr', 'cc_cons_may:max_credit_amount_apr',
       'cc_cons_may:debit_amount_may', 'cc_cons_may:credit_amount_may', 'cc_cons_may:credit_count_may',
       'cc_cons_may:debit_count_may', 'cc_cons_may:max_credit_amount_may', 'cc_cons_may:debit_amount_jun',
       'cc_cons_may:credit_amount_jun', 'cc_cons_may:credit_count_jun', 'cc_cons_may:debit_count_jun',
       'cc_cons_may:max_credit_amount_jun', 'cc_cons_may:emi_active', 'cc_cons_may:account_type_current',
       'cc_cons_may:account_type_saving', 'cc_cons_may:gender_F', 'cc_cons_may:gender_M',
        'dc_cons_may:cc_cons_jun',
        'dc_cons_may:dc_cons_jun','dc_cons_may:cc_count_apr','dc_cons_may:cc_count_may',
       'dc_cons_may:cc_count_jun', 'dc_cons_may:dc_count_apr', 'dc_cons_may:dc_count_may', 'dc_cons_may:dc_count_jun',
       'dc_cons_may:card_lim', 'dc_cons_may:investment_1', 'dc_cons_may:investment_2', 'dc_cons_may:investment_3',
       'dc_cons_may:investment_4', 'dc_cons_may:debit_amount_apr', 'dc_cons_may:credit_amount_apr',
       'dc_cons_may:debit_count_apr', 'dc_cons_may:credit_count_apr', 'dc_cons_may:max_credit_amount_apr',
       'dc_cons_may:debit_amount_may', 'dc_cons_may:credit_amount_may', 'dc_cons_may:credit_count_may',
       'dc_cons_may:debit_count_may', 'dc_cons_may:max_credit_amount_may', 'dc_cons_may:debit_amount_jun',
       'dc_cons_may:credit_amount_jun', 'dc_cons_may:credit_count_jun', 'dc_cons_may:debit_count_jun',
       'dc_cons_may:max_credit_amount_jun', 'dc_cons_may:emi_active', 'dc_cons_may:account_type_current',
       'dc_cons_may:account_type_saving', 'dc_cons_may:gender_F', 'dc_cons_may:gender_M',
        'cc_cons_jun:dc_cons_jun','cc_cons_jun:cc_count_apr','cc_cons_jun:cc_count_may',
       'cc_cons_jun:cc_count_jun', 'cc_cons_jun:dc_count_apr', 'cc_cons_jun:dc_count_may', 'cc_cons_jun:dc_count_jun',
       'cc_cons_jun:card_lim', 'cc_cons_jun:investment_1', 'cc_cons_jun:investment_2', 'cc_cons_jun:investment_3',
       'cc_cons_jun:investment_4', 'cc_cons_jun:debit_amount_apr', 'cc_cons_jun:credit_amount_apr',
       'cc_cons_jun:debit_count_apr', 'cc_cons_jun:credit_count_apr', 'cc_cons_jun:max_credit_amount_apr',
       'cc_cons_jun:debit_amount_may', 'cc_cons_jun:credit_amount_may', 'cc_cons_jun:credit_count_may',
       'cc_cons_jun:debit_count_may', 'cc_cons_jun:max_credit_amount_may', 'cc_cons_jun:debit_amount_jun',
       'cc_cons_jun:credit_amount_jun', 'cc_cons_jun:credit_count_jun', 'cc_cons_jun:debit_count_jun',
       'cc_cons_jun:max_credit_amount_jun', 'cc_cons_jun:emi_active', 'cc_cons_jun:account_type_current',
       'cc_cons_jun:account_type_saving', 'cc_cons_jun:gender_F', 'cc_cons_jun:gender_M',
        'dc_cons_jun:cc_count_apr','dc_cons_jun:cc_count_may',
       'dc_cons_jun:cc_count_jun', 'dc_cons_jun:dc_count_apr', 'dc_cons_jun:dc_count_may', 'dc_cons_jun:dc_count_jun',
       'dc_cons_jun:card_lim', 'dc_cons_jun:investment_1', 'dc_cons_jun:investment_2', 'dc_cons_jun:investment_3',
       'dc_cons_jun:investment_4', 'dc_cons_jun:debit_amount_apr', 'dc_cons_jun:credit_amount_apr',
       'dc_cons_jun:debit_count_apr', 'dc_cons_jun:credit_count_apr', 'dc_cons_jun:max_credit_amount_apr',
       'dc_cons_jun:debit_amount_may', 'dc_cons_jun:credit_amount_may', 'dc_cons_jun:credit_count_may',
       'dc_cons_jun:debit_count_may', 'dc_cons_jun:max_credit_amount_may', 'dc_cons_jun:debit_amount_jun',
       'dc_cons_jun:credit_amount_jun', 'dc_cons_jun:credit_count_jun', 'dc_cons_jun:debit_count_jun',
       'dc_cons_jun:max_credit_amount_jun', 'dc_cons_jun:emi_active', 'dc_cons_jun:account_type_current',
       'dc_cons_jun:account_type_saving', 'dc_cons_jun:gender_F', 'dc_cons_jun:gender_M',
        'cc_count_apr:cc_count_may',
       'cc_count_apr:cc_count_jun', 'cc_count_apr:dc_count_apr', 'cc_count_apr:dc_count_may', 'cc_count_apr:dc_count_jun',
       'cc_count_apr:card_lim', 'cc_count_apr:investment_1', 'cc_count_apr:investment_2', 'cc_count_apr:investment_3',
       'cc_count_apr:investment_4', 'cc_count_apr:debit_amount_apr', 'cc_count_apr:credit_amount_apr',
       'cc_count_apr:debit_count_apr', 'cc_count_apr:credit_count_apr', 'cc_count_apr:max_credit_amount_apr',
       'cc_count_apr:debit_amount_may', 'cc_count_apr:credit_amount_may', 'cc_count_apr:credit_count_may',
       'cc_count_apr:debit_count_may', 'cc_count_apr:max_credit_amount_may', 'cc_count_apr:debit_amount_jun',
       'cc_count_apr:credit_amount_jun', 'cc_count_apr:credit_count_jun', 'cc_count_apr:debit_count_jun',
       'cc_count_apr:max_credit_amount_jun', 'cc_count_apr:emi_active', 'cc_count_apr:account_type_current',
       'cc_count_apr:account_type_saving', 'cc_count_apr:gender_F', 'cc_count_apr:gender_M',
        'cc_count_may:cc_count_jun', 'cc_count_may:dc_count_apr', 'cc_count_may:dc_count_may', 'cc_count_may:dc_count_jun',
       'cc_count_may:card_lim', 'cc_count_may:investment_1', 'cc_count_may:investment_2', 'cc_count_may:investment_3',
       'cc_count_may:investment_4', 'cc_count_may:debit_amount_apr', 'cc_count_may:credit_amount_apr',
       'cc_count_may:debit_count_apr', 'cc_count_may:credit_count_apr', 'cc_count_may:max_credit_amount_apr',
       'cc_count_may:debit_amount_may', 'cc_count_may:credit_amount_may', 'cc_count_may:credit_count_may',
       'cc_count_may:debit_count_may', 'cc_count_may:max_credit_amount_may', 'cc_count_may:debit_amount_jun',
       'cc_count_may:credit_amount_jun', 'cc_count_may:credit_count_jun', 'cc_count_may:debit_count_jun',
       'cc_count_may:max_credit_amount_jun', 'cc_count_may:emi_active', 'cc_count_may:account_type_current',
       'cc_count_may:account_type_saving', 'cc_count_may:gender_F', 'cc_count_may:gender_M',
        'cc_count_jun:dc_count_apr', 'cc_count_jun:dc_count_may', 'cc_count_jun:dc_count_jun',
       'cc_count_jun:card_lim', 'cc_count_jun:investment_1', 'cc_count_jun:investment_2', 'cc_count_jun:investment_3',
       'cc_count_jun:investment_4', 'cc_count_jun:debit_amount_apr', 'cc_count_jun:credit_amount_apr',
       'cc_count_jun:debit_count_apr', 'cc_count_jun:credit_count_apr', 'cc_count_jun:max_credit_amount_apr',
       'cc_count_jun:debit_amount_may', 'cc_count_jun:credit_amount_may', 'cc_count_jun:credit_count_may',
       'cc_count_jun:debit_count_may', 'cc_count_jun:max_credit_amount_may', 'cc_count_jun:debit_amount_jun',
       'cc_count_jun:credit_amount_jun', 'cc_count_jun:credit_count_jun', 'cc_count_jun:debit_count_jun',
       'cc_count_jun:max_credit_amount_jun', 'cc_count_jun:emi_active', 'cc_count_jun:account_type_current',
       'cc_count_jun:account_type_saving', 'cc_count_jun:gender_F', 'cc_count_jun:gender_M',
        'dc_count_apr:dc_count_may', 'dc_count_apr:dc_count_jun',
       'dc_count_apr:card_lim', 'dc_count_apr:investment_1', 'dc_count_apr:investment_2', 'dc_count_apr:investment_3',
       'dc_count_apr:investment_4', 'dc_count_apr:debit_amount_apr', 'dc_count_apr:credit_amount_apr',
       'dc_count_apr:debit_count_apr', 'dc_count_apr:credit_count_apr', 'dc_count_apr:max_credit_amount_apr',
       'dc_count_apr:debit_amount_may', 'dc_count_apr:credit_amount_may', 'dc_count_apr:credit_count_may',
       'dc_count_apr:debit_count_may', 'dc_count_apr:max_credit_amount_may', 'dc_count_apr:debit_amount_jun',
       'dc_count_apr:credit_amount_jun', 'dc_count_apr:credit_count_jun', 'dc_count_apr:debit_count_jun',
       'dc_count_apr:max_credit_amount_jun', 'dc_count_apr:emi_active', 'dc_count_apr:account_type_current',
       'dc_count_apr:account_type_saving', 'dc_count_apr:gender_F', 'dc_count_apr:gender_M',
        'dc_count_may:dc_count_jun',
       'dc_count_may:card_lim', 'dc_count_may:investment_1', 'dc_count_may:investment_2', 'dc_count_may:investment_3',
       'dc_count_may:investment_4', 'dc_count_may:debit_amount_apr', 'dc_count_may:credit_amount_apr',
       'dc_count_may:debit_count_apr', 'dc_count_may:credit_count_apr', 'dc_count_may:max_credit_amount_apr',
       'dc_count_may:debit_amount_may', 'dc_count_may:credit_amount_may', 'dc_count_may:credit_count_may',
       'dc_count_may:debit_count_may', 'dc_count_may:max_credit_amount_may', 'dc_count_may:debit_amount_jun',
       'dc_count_may:credit_amount_jun', 'dc_count_may:credit_count_jun', 'dc_count_may:debit_count_jun',
       'dc_count_may:max_credit_amount_jun', 'dc_count_may:emi_active', 'dc_count_may:account_type_current',
       'dc_count_may:account_type_saving', 'dc_count_may:gender_F', 'dc_count_may:gender_M',
        'dc_count_jun:card_lim', 'dc_count_jun:investment_1', 'dc_count_jun:investment_2', 'dc_count_jun:investment_3',
       'dc_count_jun:investment_4', 'dc_count_jun:debit_amount_apr', 'dc_count_jun:credit_amount_apr',
       'dc_count_jun:debit_count_apr', 'dc_count_jun:credit_count_apr', 'dc_count_jun:max_credit_amount_apr',
       'dc_count_jun:debit_amount_may', 'dc_count_jun:credit_amount_may', 'dc_count_jun:credit_count_may',
       'dc_count_jun:debit_count_may', 'dc_count_jun:max_credit_amount_may', 'dc_count_jun:debit_amount_jun',
       'dc_count_jun:credit_amount_jun', 'dc_count_jun:credit_count_jun', 'dc_count_jun:debit_count_jun',
       'dc_count_jun:max_credit_amount_jun', 'dc_count_jun:emi_active', 'dc_count_jun:account_type_current',
       'dc_count_jun:account_type_saving', 'dc_count_jun:gender_F', 'dc_count_jun:gender_M',
        'card_limit:investment_1', 'card_limit:investment_2', 'card_limit:investment_3',
       'card_limit:investment_4', 'card_limit:debit_amount_apr', 'card_limit:credit_amount_apr',
       'card_limit:debit_count_apr', 'card_limit:credit_count_apr', 'card_limit:max_credit_amount_apr',
       'card_limit:debit_amount_may', 'card_limit:credit_amount_may', 'card_limit:credit_count_may',
       'card_limit:debit_count_may', 'card_limit:max_credit_amount_may', 'card_limit:debit_amount_jun',
       'card_limit:credit_amount_jun', 'card_limit:credit_count_jun', 'card_limit:debit_count_jun',
       'card_limit:max_credit_amount_jun', 'card_limit:emi_active', 'card_limit:account_type_current',
       'card_limit:account_type_saving', 'card_limit:gender_F', 'card_limit:gender_M',
        'investment_1:investment_2', 'investment_1:investment_3',
       'investment_1:investment_4', 'investment_1:debit_amount_apr', 'investment_1:credit_amount_apr',
       'investment_1:debit_count_apr', 'investment_1:credit_count_apr', 'investment_1:max_credit_amount_apr',
       'investment_1:debit_amount_may', 'investment_1:credit_amount_may', 'investment_1:credit_count_may',
       'investment_1:debit_count_may', 'investment_1:max_credit_amount_may', 'investment_1:debit_amount_jun',
       'investment_1:credit_amount_jun', 'investment_1:credit_count_jun', 'investment_1:debit_count_jun',
       'investment_1:max_credit_amount_jun', 'investment_1:emi_active', 'investment_1:account_type_current',
       'investment_1:account_type_saving', 'investment_1:gender_F', 'investment_1:gender_M',
        'investment_2:investment_3',
       'investment_2:investment_4', 'investment_2:debit_amount_apr', 'investment_2:credit_amount_apr',
       'investment_2:debit_count_apr', 'investment_2:credit_count_apr', 'investment_2:max_credit_amount_apr',
       'investment_2:debit_amount_may', 'investment_2:credit_amount_may', 'investment_2:credit_count_may',
       'investment_2:debit_count_may', 'investment_2:max_credit_amount_may', 'investment_2:debit_amount_jun',
       'investment_2:credit_amount_jun', 'investment_2:credit_count_jun', 'investment_2:debit_count_jun',
       'investment_2:max_credit_amount_jun', 'investment_2:emi_active', 'investment_2:account_type_current',
       'investment_2:account_type_saving', 'investment_2:gender_F', 'investment_2:gender_M',
        'investment_3:investment_4', 'investment_3:debit_amount_apr', 'investment_3:credit_amount_apr',
       'investment_3:debit_count_apr', 'investment_3:credit_count_apr', 'investment_3:max_credit_amount_apr',
       'investment_3:debit_amount_may', 'investment_3:credit_amount_may', 'investment_3:credit_count_may',
       'investment_3:debit_count_may', 'investment_3:max_credit_amount_may', 'investment_3:debit_amount_jun',
       'investment_3:credit_amount_jun', 'investment_3:credit_count_jun', 'investment_3:debit_count_jun',
       'investment_3:max_credit_amount_jun', 'investment_3:emi_active', 'investment_3:account_type_current',
       'investment_3:account_type_saving', 'investment_3:gender_F', 'investment_3:gender_M',
        'investment_4:debit_amount_apr', 'investment_4:credit_amount_apr',
       'investment_4:debit_count_apr', 'investment_4:credit_count_apr', 'investment_4:max_credit_amount_apr',
       'investment_4:debit_amount_may', 'investment_4:credit_amount_may', 'investment_4:credit_count_may',
       'investment_4:debit_count_may', 'investment_4:max_credit_amount_may', 'investment_4:debit_amount_jun',
       'investment_4:credit_amount_jun', 'investment_4:credit_count_jun', 'investment_4:debit_count_jun',
       'investment_4:max_credit_amount_jun', 'investment_4:emi_active', 'investment_4:account_type_current',
       'investment_4:account_type_saving', 'investment_4:gender_F', 'investment_4:gender_M',
        'debit_amount_apr:credit_amount_apr',
       'debit_amount_apr:debit_count_apr', 'debit_amount_apr:credit_count_apr', 'debit_amount_apr:max_credit_amount_apr',
       'debit_amount_apr:debit_amount_may', 'debit_amount_apr:credit_amount_may', 'debit_amount_apr:credit_count_may',
       'debit_amount_apr:debit_count_may', 'debit_amount_apr:max_credit_amount_may', 'debit_amount_apr:debit_amount_jun',
       'debit_amount_apr:credit_amount_jun', 'debit_amount_apr:credit_count_jun', 'debit_amount_apr:debit_count_jun',
       'debit_amount_apr:max_credit_amount_jun', 'debit_amount_apr:emi_active', 'debit_amount_apr:account_type_current',
       'debit_amount_apr:account_type_saving', 'debit_amount_apr:gender_F', 'debit_amount_apr:gender_M',
        'credit_amount_apr:debit_count_apr', 'credit_amount_apr:credit_count_apr', 'credit_amount_apr:max_credit_amount_apr',
       'credit_amount_apr:debit_amount_may', 'credit_amount_apr:credit_amount_may', 'credit_amount_apr:credit_count_may',
       'credit_amount_apr:debit_count_may', 'credit_amount_apr:max_credit_amount_may', 'credit_amount_apr:debit_amount_jun',
       'credit_amount_apr:credit_amount_jun', 'credit_amount_apr:credit_count_jun', 'credit_amount_apr:debit_count_jun',
       'credit_amount_apr:max_credit_amount_jun', 'credit_amount_apr:emi_active', 'credit_amount_apr:account_type_current',
       'credit_amount_apr:account_type_saving', 'credit_amount_apr:gender_F', 'credit_amount_apr:gender_M',
        'debit_count_apr:credit_count_apr', 'debit_count_apr:max_credit_amount_apr',
       'debit_count_apr:debit_amount_may', 'debit_count_apr:credit_amount_may', 'debit_count_apr:credit_count_may',
       'debit_count_apr:debit_count_may', 'debit_count_apr:max_credit_amount_may', 'debit_count_apr:debit_amount_jun',
       'debit_count_apr:credit_amount_jun', 'debit_count_apr:credit_count_jun', 'debit_count_apr:debit_count_jun',
       'debit_count_apr:max_credit_amount_jun', 'debit_count_apr:emi_active', 'debit_count_apr:account_type_current',
       'debit_count_apr:account_type_saving', 'debit_count_apr:gender_F', 'debit_count_apr:gender_M',
        'credit_count_apr:max_credit_amount_apr',
       'credit_count_apr:debit_amount_may', 'credit_count_apr:credit_amount_may', 'credit_count_apr:credit_count_may',
       'credit_count_apr:debit_count_may', 'credit_count_apr:max_credit_amount_may', 'credit_count_apr:debit_amount_jun',
       'credit_count_apr:credit_amount_jun', 'credit_count_apr:credit_count_jun', 'credit_count_apr:debit_count_jun',
       'credit_count_apr:max_credit_amount_jun', 'credit_count_apr:emi_active', 'credit_count_apr:account_type_current',
       'credit_count_apr:account_type_saving', 'credit_count_apr:gender_F', 'credit_count_apr:gender_M',
        'max_credit_amount_apr:debit_amount_may', 'max_credit_amount_apr:credit_amount_may', 'max_credit_amount_apr:credit_count_may',
       'max_credit_amount_apr:debit_count_may', 'max_credit_amount_apr:max_credit_amount_may', 'max_credit_amount_apr:debit_amount_jun',
       'max_credit_amount_apr:credit_amount_jun', 'max_credit_amount_apr:credit_count_jun', 'max_credit_amount_apr:debit_count_jun',
       'max_credit_amount_apr:max_credit_amount_jun', 'max_credit_amount_apr:emi_active', 'max_credit_amount_apr:account_type_current',
       'max_credit_amount_apr:account_type_saving', 'max_credit_amount_apr:gender_F', 'max_credit_amount_apr:gender_M',
        'debit_amount_may:credit_amount_may', 'debit_amount_may:credit_count_may',
       'debit_amount_may:debit_count_may', 'debit_amount_may:max_credit_amount_may', 'debit_amount_may:debit_amount_jun',
       'debit_amount_may:credit_amount_jun', 'debit_amount_may:credit_count_jun', 'debit_amount_may:debit_count_jun',
       'debit_amount_may:max_credit_amount_jun', 'debit_amount_may:emi_active', 'debit_amount_may:account_type_current',
       'debit_amount_may:account_type_saving', 'debit_amount_may:gender_F', 'debit_amount_may:gender_M',
        'credit_amount_may:credit_count_may',
       'credit_amount_may:debit_count_may', 'credit_amount_may:max_credit_amount_may', 'credit_amount_may:debit_amount_jun',
       'credit_amount_may:credit_amount_jun', 'credit_amount_may:credit_count_jun', 'credit_amount_may:debit_count_jun',
       'credit_amount_may:max_credit_amount_jun', 'credit_amount_may:emi_active', 'credit_amount_may:account_type_current',
       'credit_amount_may:account_type_saving', 'credit_amount_may:gender_F', 'credit_amount_may:gender_M',
        'credit_count_may:debit_count_may', 'credit_count_may:max_credit_amount_may', 'credit_count_may:debit_amount_jun',
       'credit_count_may:credit_amount_jun', 'credit_count_may:credit_count_jun', 'credit_count_may:debit_count_jun',
       'credit_count_may:max_credit_amount_jun', 'credit_count_may:emi_active', 'credit_count_may:account_type_current',
       'credit_count_may:account_type_saving', 'credit_count_may:gender_F', 'credit_count_may:gender_M',
        'debit_count_may:max_credit_amount_may', 'debit_count_may:debit_amount_jun',
       'debit_count_may:credit_amount_jun', 'debit_count_may:credit_count_jun', 'debit_count_may:debit_count_jun',
       'debit_count_may:max_credit_amount_jun', 'debit_count_may:emi_active', 'debit_count_may:account_type_current',
       'debit_count_may:account_type_saving', 'debit_count_may:gender_F', 'debit_count_may:gender_M',
        'max_credit_amount_may:debit_amount_jun',
       'max_credit_amount_may:credit_amount_jun', 'max_credit_amount_may:credit_count_jun', 'max_credit_amount_may:debit_count_jun',
       'max_credit_amount_may:max_credit_amount_jun', 'max_credit_amount_may:emi_active', 'max_credit_amount_may:account_type_current',
       'max_credit_amount_may:account_type_saving', 'max_credit_amount_may:gender_F', 'max_credit_amount_may:gender_M',
        'debit_amount_jun:credit_amount_jun', 'debit_amount_jun:credit_count_jun', 'debit_amount_jun:debit_count_jun',
       'debit_amount_jun:max_credit_amount_jun', 'debit_amount_jun:emi_active', 'debit_amount_jun:account_type_current',
       'debit_amount_jun:account_type_saving', 'debit_amount_jun:gender_F', 'debit_amount_jun:gender_M',
        'credit_amount_jun:credit_count_jun', 'credit_amount_jun:debit_count_jun',
       'credit_amount_jun:max_credit_amount_jun', 'credit_amount_jun:emi_active', 'credit_amount_jun:account_type_current',
       'credit_amount_jun:account_type_saving', 'credit_amount_jun:gender_F', 'credit_amount_jun:gender_M',
        'credit_count_jun:debit_count_jun',
       'credit_count_jun:max_credit_amount_jun', 'credit_count_jun:emi_active', 'credit_count_jun:account_type_current',
       'credit_count_jun:account_type_saving', 'credit_count_jun:gender_F', 'credit_count_jun:gender_M',
        'debit_count_jun:max_credit_amount_jun', 'debit_count_jun:emi_active', 'debit_count_jun:account_type_current',
       'debit_count_jun:account_type_saving', 'debit_count_jun:gender_F', 'debit_count_jun:gender_M',
        'max_credit_amount_jun:emi_active', 'max_credit_amount_jun:account_type_current',
       'max_credit_amount_jun:account_type_saving', 'max_credit_amount_jun:gender_F', 'max_credit_amount_jun:gender_M',
        'emi_active:account_type_current',
       'emi_active:account_type_saving', 'emi_active:gender_F', 'emi_active:gender_M',
        'account_type_current:account_type_saving', 'account_type_current:gender_F', 'account_type_current:gender_M',
        'account_type_saving:gender_F', 'account_type_saving:gender_M','gender_F:gender_M',])

In [70]:
#new model including the interaction terms
synergy_model = linear_model.OLS(y, x_interaction).fit()

In [73]:
synergy_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                cc_cons   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.025
Date:                Sat, 29 Feb 2020   Prob (F-statistic):              0.325
Time:                        12:17:17   Log-Likelihood:            -1.9328e+05
No. Observations:               15000   AIC:                         3.879e+05
Df Residuals:                   14337   BIC:                         3.929e+05
Df Model:                         662                                         
Covariance Type:            nonrobust                                         
===============================================================================================================
                                                  coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------
age                                           -89.4210     66.991     -1.335      0.182    -220.733      41.891
cc_cons_apr                                     3.3326      0.905      3.682      0.000       1.559       5.107
dc_cons_apr                                     4.0198      0.934      4.304      0.000       2.189       5.850
cc_cons_may                                     6.7255      1.656      4.061      0.000       3.480       9.971
dc_cons_may                                     9.1243      2.304      3.961      0.000       4.609      13.640
cc_cons_jun                                     5.4465      1.333      4.087      0.000       2.835       8.058
dc_cons_jun                                   -11.0657      2.826     -3.915      0.000     -16.605      -5.526
cc_count_apr                                   -3.4200     21.601     -0.158      0.874     -45.761      38.920
cc_count_may                                   -6.1415     46.440     -0.132      0.895     -97.169      84.886
cc_count_jun                                   21.1321     47.869      0.441      0.659     -72.697     114.962
dc_count_apr                                  311.8749    103.390      3.016      0.003     109.216     514.533
dc_count_may                                  -39.4172    119.511     -0.330      0.742    -273.674     194.840
dc_count_jun                                  -69.9841     68.790     -1.017      0.309    -204.822      64.854
card_lim                                       -0.2288      0.108     -2.112      0.035      -0.441      -0.016
investment_1                                   -0.5428      0.187     -2.904      0.004      -0.909      -0.176
investment_2                                   -0.9023      0.216     -4.178      0.000      -1.326      -0.479
investment_3                                    1.6282      0.390      4.173      0.000       0.863       2.393
investment_4                                   -1.1795      0.404     -2.922      0.003      -1.971      -0.388
debit_amount_apr                               -0.5728      0.157     -3.648      0.000      -0.881      -0.265
credit_amount_apr                              -0.9979      0.251     -3.982      0.000      -1.489      -0.507
debit_count_apr                               -20.5622     63.946     -0.322      0.748    -145.905     104.781
credit_count_apr                               24.9306    160.580      0.155      0.877    -289.827     339.689
max_credit_amount_apr                           0.3889      0.099      3.941      0.000       0.195       0.582
debit_amount_may                                0.6839      0.175      3.910      0.000       0.341       1.027
credit_amount_may                              -0.0498     

In [81]:
imp_var = synergy_model.pvalues[synergy_model.pvalues < 0.05].sort_values(ascending=False)

In [ ]:
print(imp_var.to_string())